In [ ]:
class Recipe:
    """
    Represents a recipe with its details.

    Attributes:
        name (str): The name of the recipe.
        cuisine (str): The cuisine type of the recipe.
        ingredients (dict): Dictionary of ingredients and their quantities.
        instructions (list): List of instructions for preparing the recipe.
        is_vegetarian (bool): Indicates if the recipe is vegetarian.
        servings (int): Number of servings the recipe yields.
        replacements (dict): Dictionary of ingredient replacements.
        
    """
    
    def __init__(self, name, cuisine, ingredients, instructions, is_vegetarian, servings=1, replacements=None):
        """
        Initializes a recipe with its details.

        Args:
            name (str): The name of the recipe.
            cuisine (str): The cuisine type of the recipe.
            ingredients (dict): Dictionary of ingredients and their quantities.
            instructions (list): List of instructions for preparing the recipe.
            is_vegetarian (bool): Indicates if the recipe is vegetarian.
            servings (int, optional): Number of servings the recipe yields (default is 1).
            replacements (dict, optional): Dictionary of ingredient replacements (default is None).
        """
        self.name = name
        self.cuisine = cuisine
        self.ingredients = ingredients
        self.instructions = instructions
        self.is_vegetarian = is_vegetarian
        self.servings = servings
        self.replacements = replacements if replacements else {}
        

    def adjust_servings(self, new_servings):
        """Adjusts the quantities of ingredients based on new servings.

        Args:
            new_servings (int): New number of servings.

        Returns:
            dict: Adjusted ingredients with updated quantities.
        
        Raises:
            ValueError: If new_servings is less than or equal to zero.
        """
        if new_servings <= 0:
            raise ValueError("Servings must be greater than zero.")
        
        factor = new_servings / self.servings
        adjusted_ingredients = {ingredient: (quantity * factor,unit) for ingredient, (quantity,unit) in self.ingredients.items()}
        return adjusted_ingredients

    def suggest_replacement(self, ingredient):
        """Provides a list of suggested replacements for a given ingredient."""
        return self.replacements.get(ingredient.lower(), [])

    def format_ingredients(self):
        """Formats the ingredients of the recipe into a string.

        Returns:
            str: Formatted string listing all ingredients.
        """
        formatted_ingredients = "\n".join([f"{i + 1}. {ingredient}: {quantity} {unit}" for i, (ingredient, (quantity, unit)) in enumerate(self.ingredients.items())])
        return formatted_ingredients

    def format_instructions(self):
        """ Formats the instructions of the recipe into a string.

        Returns:
            str: Formatted string listing all instructions.
        """
        formatted_instructions = "\n".join([f"Step {i + 1}: {step}" for i, step in enumerate(self.instructions)])
        return formatted_instructions

    def __str__(self):
        """
        Returns a string representation of the recipe.
        """
        return f"Recipe: {self.name}\nCuisine: {self.cuisine}\nServings: {self.servings}\nType: {'Vegetarian' if self.is_vegetarian else 'Non-Vegetarian'}\nIngredients:\n{self.format_ingredients()}\nInstructions:\n{self.format_instructions()}\n"

   


class RecipeBook:
    """
    Represents a collection of recipes and provides operations to manage them.

    Attributes:
        recipes (list): List of Recipe objects.
        program_ratings (list): List of ratings given to the program.
        program_comments (list): List of comments left for the program.
        USERNAME (str): Username for authentication.
        PASSWORD (str): Password for authentication.
    """
    def __init__(self):
        self.recipes = []
        self.program_ratings = []
        self.program_comments = []
        self.USERNAME = "admin"
        self.PASSWORD = "password"

    def authenticate(self):
        username_input = input("Enter username: ")
        password_input = input("Enter password: ")
        if username_input == self.USERNAME and password_input == self.PASSWORD:
            return True
        else:
            print("Invalid username or password.")
            return False
        
        
    def load_from_file(self):
        """ Loads recipes, program ratings, and comments from a file named 'recipes.txt'.
        """
        self.recipes = []
        self.program_ratings = []
        self.program_comments = []

        try:
            with open('recipes.txt', 'r') as file:
                lines = file.readlines()

            i = 0
            while i < len(lines):
                line = lines[i].strip()
                if lines[i].strip() == 'Recipe:':
                    name = lines[i + 1].strip()
                    cuisine = lines[i + 2].strip()
                    is_vegetarian = lines[i + 3].strip() == 'True'
                    servings = int(lines[i + 4].strip())

                    # Load ingredients
                    ingredients = {}
                    i += 5
                    while lines[i].strip() != 'Ingredients:':
                        i += 1
                    i += 1
                    while lines[i].strip() != 'Instructions:':
                        ingredient_data = lines[i].strip().split(':')
                        ingredient_name = ingredient_data[0].strip()
                        quantity, unit = ingredient_data[1].strip().split()
                        ingredients[ingredient_name] = (float(quantity), unit)
                        i += 1

                    # Load instructions
                    instructions = []
                    i += 1
                    while lines[i].strip() != 'Replacements:':
                        instructions.append(lines[i].strip())
                        i += 1

                    # Load replacements
                    replacements = {}
                    i += 1
                    while i < len(lines) and lines[i].strip() != '':
                        replacement_data = lines[i].strip().split(':')
                        original_ingredient = replacement_data[0].strip()
                        replacement_info = replacement_data[1].strip().split()
                        quantity = float(replacement_info[0])
                        unit = replacement_info[1]
                        replacement_name = ' '.join(replacement_info[2:])
                        replacements[original_ingredient] = (quantity, unit, replacement_name)
                        i += 1

                    recipe = Recipe(name, cuisine, ingredients, instructions, is_vegetarian, servings, replacements)
                    self.recipes.append(recipe)
                
                elif line == '=== Program Ratings ===':
                    i += 1
                    while i < len(lines) and lines[i].strip() != '=== Program Comments ===':
                        line = lines[i].strip()
                        if line:  # Check if line is not empty
                            try:
                                self.program_ratings.append(float(line))
                            except ValueError:
                                print(f"Invalid program rating found: {line}. Skipping.")
                        i += 1
                elif line == '=== Program Comments ===':
                    i += 1
                    while i < len(lines)and not lines[i].strip().startswith('==='):
                        line = lines[i].strip()
                        if line:
                            self.program_comments.append(line)
                        i += 1
                else:
                    i += 1

            print("Recipes loaded successfully from file.")
            print("Program ratings loaded successfully.")
            print("Program comments loaded successfully.")
        except FileNotFoundError:
            print("Recipe file not found. Starting with an empty recipe book.")

    def add_recipe(self):
        """Adds a new recipe to the recipe book after authentication."""
        if self.authenticate():
            name = input("Enter the recipe name: ")
            cuisine = input("Enter the cuisine: ")
            is_vegetarian = input("Is the recipe vegetarian? (yes/no): ").strip().lower() == 'yes'

            servings = int(input("Enter the number of servings: "))
            ingredients = {}
            while True:
                ingredient = input("Enter an ingredient (or type 'done' to finish): ")
                if ingredient.lower() == 'done':
                    break
                while True:
                    try:
                        quantity = float(input(f"Enter the quantity for {ingredient}: "))
                        if quantity <= 0:
                            print("Quantity must be greater than zero. Please enter again.")
                            continue
                        unit = input(f"Enter the unit for {ingredient}: ")
                        ingredients[ingredient] = (quantity, unit)
                        break
                    except ValueError:
                        print("Invalid quantity. Please enter a valid number.")

            instructions = []
            print("Enter the instructions for the recipe step by step. Enter 'done' when you have finished entering instructions.")
            step_number = 1
            while True:
                step = input(f"Step {step_number}: ").strip()
                if step.lower() == 'done':
                    break
                instructions.append(step)
                step_number += 1

            replacements = {}
            suggest_replacements = input("Do you want to suggest replacements for any ingredients? (yes/no): ").strip().lower() == 'yes'
            if suggest_replacements:
                while True:
                    original_ingredient = input("Enter the original ingredient name for replacement (or type 'done' to finish): ")
                    if original_ingredient.lower() == 'done':
                        break
                    replacement_name = input(f"Enter the replacement ingredient name for {original_ingredient}: ")
                    while True:
                        try:
                            quantity = float(input(f"Enter the quantity for {replacement_name}: "))
                            if quantity <= 0:
                                print("Quantity must be greater than zero. Please enter again.")
                                continue
                            unit = input(f"Enter the unit for {replacement_name}: ")
                            replacements[original_ingredient] = (quantity, unit, replacement_name)
                            break
                        except ValueError:
                            print("Invalid quantity. Please enter a valid number.")

        recipe = Recipe(name, cuisine, ingredients, instructions, is_vegetarian, servings, replacements)
        self.recipes.append(recipe)
        print("Recipe added successfully.")
        self.save_to_file()  # Save recipes to file after adding

        # Reload recipes from file to synchronize
        self.load_from_file()
        
    def edit_recipe(self):
        """Allows editing an existing recipe in the recipe book after authentication."""
        if self.authenticate():
            recipe_name = input("Enter the name of the recipe you want to edit: ").strip().lower()
        
            found_recipes = [recipe for recipe in self.recipes if recipe.name.lower() == recipe_name]
            if not found_recipes:
                print(f"Recipe '{recipe_name}' not found.")
                return
        
            recipe_to_edit = found_recipes[0]
            print(f"Editing recipe: {recipe_to_edit.name}")
        
            while True:
                print("\nSelect what you want to edit:")
                print("1. Name")
                print("2. Cuisine")
                print("3. Vegetarian")
                print("4. Servings")
                print("5. Ingredients")
                print("6. Instructions")
                print("7. Replacements")
                print("8. Save and Exit")
            
                choice = input("Enter your choice (1-8): ").strip()
            
                if choice == '1':
                    new_name = input(f"Enter new name (current: {recipe_to_edit.name}): ").strip()
                    recipe_to_edit.name = new_name if new_name else recipe_to_edit.name
            
                elif choice == '2':
                    new_cuisine = input(f"Enter new cuisine (current: {recipe_to_edit.cuisine}): ").strip()
                    recipe_to_edit.cuisine = new_cuisine if new_cuisine else recipe_to_edit.cuisine
            
                elif choice == '3':
                    new_is_vegetarian = input(f"Is the recipe vegetarian? (yes/no) (current: {'yes' if recipe_to_edit.is_vegetarian else 'no'}): ").strip().lower()
                    if new_is_vegetarian == 'yes':
                        recipe_to_edit.is_vegetarian = True
                    elif new_is_vegetarian == 'no':
                        recipe_to_edit.is_vegetarian = False
            
                elif choice == '4':
                    try:
                        new_servings = int(input(f"Enter new servings (current: {recipe_to_edit.servings}): ").strip())
                        recipe_to_edit.servings = new_servings
                    except ValueError:
                        print("Invalid input. Servings must be a number.")
            
                elif choice == '5':
                    new_ingredients = {}
                    print("Enter updated ingredients. Press Enter to keep current values.")
                    for ingredient, (quantity, unit) in recipe_to_edit.ingredients.items():
                        print(f"\nCurrent {ingredient}: {quantity} {unit}")
                        edit_option = input(f"Do you want to edit {ingredient}? (yes/no): ").strip().lower()

                        if edit_option == 'yes':
                            print("Select option:")
                            print("1. Change ingredient name")
                            print("2. Change quantity")
                            print("3. Change unit")
                            print("4. Remove ingredient")
                            option = input("Enter your choice (1/2/3/4): ").strip()

                            if option == '1':
                                new_ingredient_name = input("Enter new ingredient name: ").strip()
                                new_ingredients[new_ingredient_name] = (quantity, unit)
                            elif option == '2':
                                try:
                                    new_quantity = float(input("Enter new quantity: ").strip())
                                    new_ingredients[ingredient] = (new_quantity, unit)
                                except ValueError:
                                    print("Invalid quantity format. Keeping current value.")
                                new_ingredients[ingredient] = (quantity, unit)
                            elif option == '3':
                                new_unit = input("Enter new unit: ").strip()
                                new_ingredients[ingredient] = (quantity, new_unit)
                            elif option == '4':
                                continue  # Skip this ingredient, effectively removing it
                            else:
                                print("Invalid option. Keeping current value.")
                                new_ingredients[ingredient] = (quantity, unit)
                        else:
                            new_ingredients[ingredient] = (quantity, unit)
                        
                    add_new_ingredient = input("Do you want to add a new ingredient? (yes/no): ").strip().lower()
                    if add_new_ingredient == 'yes':
                        while True:
                            new_ingredient_name = input("Enter new ingredient name: ").strip()
                            try:
                                new_quantity = float(input("Enter quantity for new ingredient: ").strip())
                                new_unit = input("Enter unit for new ingredient: ").strip()
                                new_ingredients[new_ingredient_name] = (new_quantity, new_unit)
                                break  # Break the loop if successful
                            except ValueError:
                                print("Invalid quantity format. Please enter a valid quantity.")
                
                    recipe_to_edit.ingredients = new_ingredients
            
                elif choice == '6':
                    new_instructions = []
                    print("Enter updated instructions. Enter 'done' when finished. Press Enter to keep current values.")
                    for i, step in enumerate(recipe_to_edit.instructions, start=1):
                        new_step = input(f"Enter new step {i} (current: {step}): ").strip() or step
                        new_instructions.append(new_step)

                    while True:
                        additional_step = input("Do you want to add an additional step? (yes/no): ").strip().lower()
                        if additional_step == 'yes':
                            new_step = input("Enter new step: ").strip()
                            new_instructions.append(new_step)
                        elif additional_step == 'no':
                            break
                        else:
                            print("Invalid input. Please enter 'yes' or 'no'.")
                
                    recipe_to_edit.instructions = new_instructions
            
                elif choice == '7':
                    new_replacements = {}
                    print("Enter updated replacements. Enter 'done' when finished. Press Enter to keep current values.")
                    for original, (quantity, unit, replacement_name) in recipe_to_edit.replacements.items():
                        new_replacement_name = input(f"Enter new replacement for {original} (current: {replacement_name}): ").strip() or replacement_name
                        new_quantity = input(f"Enter new quantity for {original} (current: {quantity}): ").strip() or str(quantity)
                        new_unit = input(f"Enter new unit for {original} (current: {unit}): ").strip() or unit

                        # If user wants to remove this replacement
                        remove_replacement = input(f"Do you want to remove the replacement for {original}? (yes/no): ").strip().lower()
                        if remove_replacement == 'yes':
                            continue  # Skip adding this replacement to new_replacements
                        else:
                            new_replacements[original] = (float(new_quantity), new_unit, new_replacement_name)
                
                    while True:
                        add_new_replacement = input("Do you want to add a new replacement? (yes/no): ").strip().lower()
                        if add_new_replacement == 'yes':
                            original = input("Enter the original ingredient name for replacement: ")
                            replacement_name = input(f"Enter the replacement ingredient name for {original}: ")
                            while True:
                                try:
                                    quantity = float(input(f"Enter the quantity for {replacement_name}: "))
                                    if quantity <= 0:
                                        print("Quantity must be greater than zero. Please enter again.")
                                        continue
                                    unit = input(f"Enter the unit for {replacement_name}: ")
                                    new_replacements[original] = (quantity, unit, replacement_name)
                                    break
                                except ValueError:
                                    print("Invalid quantity. Please enter a valid number.")
                        elif add_new_replacement == 'no':
                            break
                        else:
                            print("Invalid input. Please enter 'yes' or 'no'.")
                
                    recipe_to_edit.replacements = new_replacements
            
                elif choice == '8':
                    print("Saving changes and exiting.")
                    self.save_to_file()  # Save recipes to file after editing
                    break
            
                else:
                    print("Invalid choice. Please enter a valid number.")

            print("Recipe updated successfully.")           
                             

    def delete_recipe(self):
        """Deletes a recipe from the recipe book after authentication."""
        if self.authenticate():
            name = input("Enter the name of the recipe to delete: ")
            recipe = self.find_recipe_by_name(name)
            if recipe:
                self.recipes.remove(recipe)
                print(f"Recipe '{name}' deleted successfully.")
                self.save_to_file()  # Save recipes to file after deleting
            else:
                print(f"Recipe '{name}' not found.")
                
    def rate_program(self):
        """Allows the user to rate the program and optionally leave a comment."""
        rating = input("Enter your rating for the program (1-5): ")
        try:
            rating = float(rating)
            if 1 <= rating <= 5:
                self.program_ratings.append(rating)
                comment = input("Enter your comment (optional): ")
                if comment:
                    self.program_comments.append(comment)
                print("Thank you for your feedback!")
                self.save_to_file()  # Save program ratings and comments
            else:
                print("Invalid rating. Please enter a rating between 1 and 5.")
        except ValueError:
            print("Invalid input. Please enter a valid number.")

    def display_program_ratings_and_comments(self):
        """Displays the average program rating and all comments left for the program."""
        if not self.program_ratings:
            print("No ratings yet.")
        else:
            print(f"Average Program Rating: {sum(self.program_ratings) / len(self.program_ratings)}")

        if not self.program_comments:
            print("No comments yet.")
        else:
            print("Program Comments:")
            for comment in self.program_comments:
                print(comment)
        
    def save_to_file(self):
        """Saves recipes, program ratings, and comments to a file named 'recipes.txt'"""
        with open("recipes.txt", "w") as file:
            for recipe in self.recipes:
                file.write("Recipe:\n")
                file.write(f"{recipe.name}\n")
                file.write(f"{recipe.cuisine}\n")
                file.write(f"{recipe.is_vegetarian}\n")
                file.write(f"{recipe.servings}\n")

                # Save ingredients
                file.write("Ingredients:\n")
                for ingredient, (quantity, unit) in recipe.ingredients.items():
                    file.write(f"{ingredient}: {quantity} {unit}\n")

                # Save instructions
                file.write("Instructions:\n")
                for step in recipe.instructions:
                    file.write(f"{step}\n")

                # Save replacements
                file.write("Replacements:\n")
                for original, (quantity, unit, replacement_name) in recipe.replacements.items():
                    file.write(f"{original}: {quantity} {unit} {replacement_name}\n")

                file.write("\n")

            # Save program ratings
            file.write("=== Program Ratings ===\n")
            for rating in self.program_ratings:
                file.write(f"{rating}\n")
            file.write("\n")

            # Save program comments
            file.write("=== Program Comments ===\n")
            for comment in self.program_comments:
                file.write(f"{comment}\n")
            file.write("\n") 
                    
    def find_recipe_by_name(self, name):
        """
        Searches for a recipe by name in the recipe book.
        Args:
            name (str): The name of the recipe to search for."""
        found_recipe = None
        for recipe in self.recipes:
            if recipe.name.lower() == name.lower():
                found_recipe = recipe
                break
        
        if found_recipe:
            self.display_recipe_options(found_recipe)
        else:
            print(f"Recipe '{name}' not found.")
            
    def find_recipes_by_cuisine(self, cuisine):
        """Searches for recipes by cuisine in the recipe book.

        Args:
            cuisine (str): The cuisine to search for.

        Returns:
            list: List of Recipe objects matching the cuisine.
        """
        found_recipes = [recipe for recipe in self.recipes if recipe.cuisine.lower() == cuisine.lower()]
    
        if found_recipes:
            print(f"Recipes found for cuisine '{cuisine}':")
            for i, recipe in enumerate(found_recipes, start=1):
                print(f"{i}. {recipe.name}")
        
            while True:
                choice = input("Enter the number of the recipe to find details, or 'done' to exit: ").strip()
            
                if choice.lower() == 'done':
                    return None
                elif choice.isdigit() and 1 <= int(choice) <= len(found_recipes):
                    recipe_to_find = found_recipes[int(choice) - 1]
                    self.display_recipe_options(recipe_to_find)
                    return recipe_to_find
                else:
                    print("Invalid choice. Please enter a valid number or 'done' to exit.")
    
        else:
            print(f"No recipes found for cuisine '{cuisine}'.")

    def find_recipes_by_ingredient(self, ingredient):
        """Searches for recipes by ingredient in the recipe book.

        Args:
            ingredient (str): The ingredient to search for.

        Returns:
            list: List of Recipe objects containing the ingredient.
        """
        found_recipes = [recipe for recipe in self.recipes if ingredient.lower() in [ing.lower() for ing in recipe.ingredients.keys()]]
        
        if found_recipes:
            print(f"Recipes found for ingredient '{ingredient}':")
            for i, recipe in enumerate(found_recipes, start=1):
                print(f"{i}. {recipe.name}")

            while True:
                choice = input("Enter the number of the recipe to find details, or 'done' to exit: ").strip()
                
                if choice.lower() == 'done':
                    return None
                elif choice.isdigit() and 1 <= int(choice) <= len(found_recipes):
                    recipe_to_find = found_recipes[int(choice) - 1]
                    self.display_recipe_options(recipe_to_find)
                    return recipe_to_find
                else:
                    print("Invalid choice. Please enter a valid number or 'done' to exit.")
        
        else:
            print(f"No recipes found for ingredient '{ingredient}'.")
            
    def display_recipe_options(self, recipe):
        """Displays options for interacting with a specific recipe."""
        
        print(recipe)

        adjust_servings_input = input("Do you want to adjust servings? (yes/no): ").strip().lower()
        if adjust_servings_input == 'yes':
            new_servings = int(input("Enter the number of servings: "))
            adjusted_ingredients = recipe.adjust_servings(new_servings)
            print("Adjusted Ingredients:")
            for ingredient, (quantity, unit) in adjusted_ingredients.items():
                print(f"{ingredient}: {quantity} {unit}")

        while True:
            suggest_replacement_input = input("Do you want to suggest replacements? (yes/no): ").strip().lower()
            if suggest_replacement_input == 'yes':
                ingredient_for_replacement = input("Enter the ingredient name for replacement: ").strip().lower()
                replacements = recipe.suggest_replacement(ingredient_for_replacement)
                if replacements:
                    print(f"Replacements for {ingredient_for_replacement}:")
                    for replacement in replacements:
                        print(replacement)
                else:
                    print(f"No replacements found for {ingredient_for_replacement}.")
            elif suggest_replacement_input == 'no':
                break  # Exit the loop if user doesn't want to suggest more replacements
            else:
                print("Invalid input. Please enter 'yes' or 'no'.")

        print("Exiting suggestion of replacements.")

    def list_recipes(self):
        """Lists all the names of recipes in the recipe book."""
        if not self.recipes:
            print("No recipes found.")
        else:
            print("The list of recipes:")
            for recipe in self.recipes:
                print(recipe.name)
    
    def menu(self):
        """Displays the main menu of operations for the recipe book."""
        print("Let's get started.....")
        print("1. Add Recipe")
        print("2. Edit Recipe")
        print("3. Delete Recipe")
        print("4. Find Recipe by Name")
        print("5. Find Recipes by Cuisine")
        print("6. Find Recipes by Ingredient")
        print("7. List All Names of Recipes")
        print("8. Rate the Program")
        print("9. Display Program Ratings and Comments")
        print("0. Exit")
        print("=======================")

    def run(self):
        """Runs the recipe book program, allowing users to perform operations."""
        
        print("****************** WELCOME TO PAVI'S KITCHEN ***********************",flush=True)
        self.load_from_file()
        while True:
            self.menu()
            choice = input("Enter your choice: ")
            if choice == '1':
                self.add_recipe()
            elif choice == '2':
                self.edit_recipe()
            elif choice == '3':
                self.delete_recipe()
            elif choice == '4':
                name = input("Enter the name of the recipe to find: ")
                recipe = self.find_recipe_by_name(name)
                if recipe:
                    print(recipe)
                else:
                    print("Thank you!have a great day!")
            elif choice == '5':
                cuisine = input("Enter the cuisine to find recipes: ")
                recipes = self.find_recipes_by_cuisine(cuisine)
                print("Thank you ! have a great day!")
            elif choice == '6':
                ingredient = input("Enter the ingredient to find recipes: ")
                recipes = self.find_recipes_by_ingredient(ingredient)
                print("Thank you ! have a great day!")
            elif choice == '7':
                self.list_recipes()
            elif choice == '8':
                self.rate_program()
            elif choice == '9':
                self.display_program_ratings_and_comments()
            elif choice == '0':
                print("Exiting program... Goodbye!")
                break
            else:
                print("Invalid choice. Please enter a number from the menu.")

if __name__ == "__main__":
    recipe_book = RecipeBook()
    recipe_book.run()

    
        

****************** WELCOME TO PAVI'S KITCHEN ***********************
Recipes loaded successfully from file.
Program ratings loaded successfully.
Program comments loaded successfully.
Let's get started.....
1. Add Recipe
2. Edit Recipe
3. Delete Recipe
4. Find Recipe by Name
5. Find Recipes by Cuisine
6. Find Recipes by Ingredient
7. List All Names of Recipes
8. Rate the Program
9. Display Program Ratings and Comments
0. Exit
Enter your choice: 9
Average Program Rating: 4.5
Program Comments:
really very user friendly
very easy to use
Let's get started.....
1. Add Recipe
2. Edit Recipe
3. Delete Recipe
4. Find Recipe by Name
5. Find Recipes by Cuisine
6. Find Recipes by Ingredient
7. List All Names of Recipes
8. Rate the Program
9. Display Program Ratings and Comments
0. Exit
